In [581]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from matchup_generator import generate_matchups

In [582]:
training_df = generate_matchups(10)
training_df

/Users/anirudh/UIUC/go-mama-27/personal-projects/poke-predictor/ml-model/db.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,p1_id,p1_name,p1_hp,p1_attack,p1_defense,p1_sp_attack,p1_sp_defense,p1_speed,p1_primary_type,p1_secondary_type,...,opp6_name,opp6_hp,opp6_attack,opp6_defense,opp6_sp_attack,opp6_sp_defense,opp6_speed,opp6_primary_type,opp6_secondary_type,winner
0,114,tangela,65,55,115,100,40,60,grass,None,...,kangaskhan,105,95,80,40,80,90,normal,None,2
1,53,persian,65,70,60,65,65,115,normal,None,...,oddish,45,50,55,75,65,30,grass,poison,2
2,100,voltorb,40,30,50,55,55,100,electric,None,...,muk,105,105,75,65,100,50,poison,None,2
3,48,venonat,60,55,50,40,55,45,bug,poison,...,weedle,40,35,30,20,20,50,bug,poison,2
4,58,growlithe,55,70,45,70,50,60,fire,None,...,lickitung,90,55,75,60,75,30,normal,None,1
5,104,cubone,50,50,95,40,50,35,ground,None,...,koffing,40,65,95,60,45,35,poison,None,1
6,84,doduo,35,85,45,35,35,75,normal,flying,...,nidorino,61,72,57,55,55,65,poison,None,2
7,30,nidorina,70,62,67,55,55,56,poison,None,...,poliwhirl,65,65,65,50,50,90,water,None,1
8,145,zapdos,90,90,85,125,90,100,electric,flying,...,snorlax,160,110,65,65,110,30,normal,None,2
9,10,caterpie,45,30,35,20,20,45,bug,None,...,arcanine,90,110,80,100,80,95,fire,None,2


In [583]:
X = training_df.drop(columns=["winner"])
y = training_df["winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [584]:
numeric_cols_x = X_train.select_dtypes(include='number').columns

sc = StandardScaler()
X_train_numeric = sc.fit_transform(X_train[numeric_cols_x])
X_test_numeric = sc.transform(X_test[numeric_cols_x])

In [585]:
X_train = torch.from_numpy(X_train_numeric.astype(np.float32))
X_test = torch.from_numpy(X_test_numeric.astype(np.float32))

''' 
    loss function needs values 0 and 1. 
    will make p1 win equal to 0 and p2 win equal to 1 
    (were originally 1 and 2 respectively)
'''
y_train = torch.from_numpy((y_train.to_numpy() - 1).astype(np.float32))
y_test = torch.from_numpy((y_test.to_numpy() - 1).astype(np.float32))

In [586]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

y_train = y_train.float()
y_test = y_test.float()

In [587]:
class LogisticRegression(nn.Module):
    def __init__(self, n):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n, 1)
        
    def forward(self, x):
        return self.linear(x)

In [588]:
n = X_train.shape[1]
model = LogisticRegression(n)

In [589]:
learning_rate = 0.01
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [590]:
num_epochs = 100

for epoch in range(num_epochs):
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 10, loss = 0.1860
epoch: 20, loss = 0.0678
epoch: 30, loss = 0.0350
epoch: 40, loss = 0.0231
epoch: 50, loss = 0.0174
epoch: 60, loss = 0.0141
epoch: 70, loss = 0.0120
epoch: 80, loss = 0.0104
epoch: 90, loss = 0.0092
epoch: 100, loss = 0.0081


In [591]:
with torch.no_grad():
    y_pred_logits = model(X_test).squeeze()
    y_pred_probs = torch.sigmoid(y_pred_logits)
    y_pred_cls = (y_pred_probs >= 0.5).float()
    acc = (y_pred_cls == y_test).float().mean()
    print(f'accuracy = {acc:.4f}')

accuracy = 0.5000
